In [0]:
import sys
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from sklearn import metrics


if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

Given train.dat file conatins inputs of documents and frequency of terms in all the documnets. Read the data as csr matrix where every row is a document and every column is termID and the cell contains the frequency of the termID

In [0]:
def convert_dat_csrmatrix(filename):
    
    row_id = []
    column_id = []
    frequency = []
    
    file = open(filename,'r')
    
    for i, eachline in enumerate(file):
        eachline = eachline.strip()
        data = eachline.split()
        
        for index in range(len(data)):
            
            if index % 2 == 0:
                column_id.append(int(data[index]))
                frequency.append(int(data[index + 1]))
                row_id.append(i)
    
    matrix = csr_matrix((frequency, (row_id, column_id)), dtype = np.float)
    
    return matrix

We have the term frequency , inorder find the the frequency and importance of the terms while clustering find TF-IDF = Term Frequency (TF) * Inverse Document Frequency (IDF) using L2 norm : The cosine similarity between two vectors is their dot product when l2 norm has been applied on the documents 

In [0]:
def calculate_tfidf(matrix):
    
    from sklearn.feature_extraction.text import TfidfTransformer
    transformer = TfidfTransformer(norm='l2', use_idf=True,smooth_idf=False)
    dataset_tfidf = transformer.fit_transform(matrix)
    
    return dataset_tfidf


Inorder reduce the computation time and To avoid risk of overfitting model result in terrible out of sample performance we need to reduce the dimenstionality of the matrix to n columns

In [0]:
def reduce_dimensionality(matrix_tfid):
    
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=150)
    reduced = svd.fit_transform(matrix_tfid)
    
    return reduced


Choosing 2 random rows data and assign them as initial centroids

In [0]:
def initialize_centroids(reduced_matrix,K):
    
    init_centroids = reduced_matrix[np.random.choice(reduced_matrix.shape[0], size=K, replace=False),:]

    return init_centroids

Using euclidean_distance measure to find the optimal centroids - that is distance between the centroids and given document data points

In [0]:
def calculate_euclidean_dist(centroid_points,reduced_matrix_rows):
    
    return np.sqrt(np.sum(np.power(centroid_points - reduced_matrix_rows,2)))

In [0]:
def calculate_optimalcentroids(reduced_matrix,initial_centroids,initial_clusters,K,rows):
    
    optimal_centroids = True
    while optimal_centroids:
        optimal_centroids = False
        for row in range(rows):
            d = np.inf
            index = -1
            for cluster_index in range(K):
                euclid_dist = calculate_euclidean_dist(initial_centroids[cluster_index,:],reduced_matrix[row,:])
                if euclid_dist < d :
                    d = euclid_dist
                    index = cluster_index
            if initial_clusters[row,0] != index:
                optimal_centroids = True
            initial_clusters[row,:] = index , d**2
            
        for i in range(K):
            cluster_datapoints = reduced_matrix[np.nonzero(initial_clusters[:,0].A == i)[0]]
            initial_centroids[i,:] = np.mean(cluster_datapoints,axis=0)
    
        
    return initial_centroids,initial_clusters   

KMeans Algorithm with K = 2, that gives optimal centroids of the two clusters found

In [0]:
def KMeans_clustering(reduced_matrix,K):
    
    rows = np.shape(reduced_matrix)[0]
    initial_clusters = np.mat(np.zeros((rows,2)))
    initial_centroids = initialize_centroids(reduced_matrix,K)
    
    initial_centroids,initial_clusters = calculate_optimalcentroids(reduced_matrix,initial_centroids,initial_clusters,K,rows)
    
    
    return initial_centroids,initial_clusters

In [0]:
def find_clusters(reduced_matrix,K,iterations,centroids,clusters,minimum_SSE):
    
    for i in range(iterations):
        
        initial_centroids,initial_clusters = KMeans_clustering(reduced_matrix,K)
        
        cal_SSE = np.sum(initial_clusters,axis=0)[0,1]
        
        if cal_SSE < minimum_SSE:
            minimum_SSE = cal_SSE

            store_centroids = initial_centroids
            store_clusters = initial_clusters
    
    centroids.append(store_centroids[0].tolist()[0])
    centroids.append(store_centroids[1].tolist()[0])
    clusters = store_clusters
    
    return centroids,clusters,minimum_SSE

In [0]:
def select_cluster_withmaxSSE(centroids,clusters,maximum_SSE,index):
    
    for c in range(len(centroids)):
        cal_SSE = np.sum(clusters[np.nonzero(clusters[:,0] == c)[0]])
        if cal_SSE > maximum_SSE:
            index = c
            maximum_SSE = cal_SSE
            
    return index,maximum_SSE

In [0]:
def store_cluster_withminSSE(reduced_matrix,clusters,minimum_SSE,K,iterations,index):
    
    for i in range(iterations):
            store_clusters = reduced_matrix[np.nonzero(clusters[:,0] == index)[0]]
            initial_centroids,initial_clusters = KMeans_clustering(store_clusters, 2)
            cal_SSE = np.sum(clusters[:,1],axis=0)
            
            if cal_SSE < minimum_SSE:
                minimum_SSE = cal_SSE
                temp_centroids = initial_centroids.copy()
                temp_clusters = initial_clusters.copy()
                
    return temp_centroids,temp_clusters

In [0]:
def update_cluster_details(temp_centroids,temp_clusters,index,centroids,clusters):
    
    temp_clusters[np.nonzero(temp_clusters[:,0]==1)[0],0] = len(centroids)
    temp_clusters[np.nonzero(temp_clusters[:,0]==0)[0],0] = index

    clusters[np.nonzero(clusters[:,0] == index)[0],:] = temp_clusters
    centroids[index]=temp_centroids[0].tolist()[0]
    centroids.append(temp_centroids[1].tolist()[0])
    
    return centroids,clusters

Find two optimal clusters and run the K means algorithm till we reach K  =7 where we always bisect the larger SSE cluster and append all smaller SSE clusters to our foinal list

In [0]:
def Bisecting_KMeans(reduced_matrix,K,iterations):
    
    rows = reduced_matrix.shape[0]
    columns = reduced_matrix.shape[1]
    centroids = []
    clusters = np.mat(np.zeros((rows,2)))
    minimum_SSE = np.inf
    
    centroids,clusters,minimum_SSE = find_clusters(reduced_matrix,2,iterations,centroids,clusters,minimum_SSE)
    
    minimum_SSE = np.inf
    
    while len(centroids) < K :
        index = -1
        maximum_SSE = -1
        
        index,maximum_SSE = select_cluster_withmaxSSE(centroids,clusters,maximum_SSE,index)
        temp_centroids,temp_clusters = store_cluster_withminSSE(reduced_matrix,clusters,minimum_SSE,K,iterations,index)
        centroids,clusters = update_cluster_details(temp_centroids,temp_clusters,index,centroids,clusters)
 
    return centroids,clusters

At K = 7 Save the document ID and cluster ID as format.dat file to validate the NMI score in Kaggle

In [0]:
def save_clusterinfo(reduced_matrix,list_of_clusters):
    doc_id = []
    for i in range(1,reduced_matrix.shape[0]+1):
        doc_id.append(i)

    output = open("format_final2.dat", "w")
    print("{0}{1}{2}".format('ItemID',',','ClusterID'),file=output)
    for document_id,value in zip(doc_id,list_of_clusters[:, 0]+1):
        print("{0}{1}{2}".format(document_id,',',int(value.A[0][0])), file=output)

    output.close()

In [0]:
def run_bi_kmeans_plot_metrics(reduced_matrix,K,iterations):

    scores = []
    K_values = []
    cluster_labels = []
    
    for K in range(3,22,2):

        list_of_centroids,list_of_clusters = Bisecting_KMeans(reduced_matrix, K, iterations)

        if K == 7:
            save_clusterinfo(reduced_matrix,list_of_clusters) 

        for value in list_of_clusters[:, 0]+1:
            cluster_labels.append((int(value.A[0][0])))
        
        K_values.append(K)
        scores.append(metrics.silhouette_score(reduced_matrix, cluster_labels))
        cluster_labels = []

    plt.plot(K_values, scores)
    plt.xticks(scores, scores)
    plt.xlabel('Number of Clusters k')
    plt.ylabel('Silhouette Score')
    plt.grid(linestyle='dotted')
    plt.show()

In [0]:
matrix = convert_dat_csrmatrix('train.dat')
print(matrix.shape)

matrix_tfid = calculate_tfidf(matrix)
print("tfidf done")

reduced_matrix = reduce_dimensionality(matrix_tfid)
print("Dataset after reducing the features", reduced_matrix.shape)

run_bi_kmeans_plot_metrics(reduced_matrix,7,20)
print(list_of_clusters)
len(list_of_clusters)

(8580, 126356)
tfidf done


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:1466: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


Dataset after reducing the features (8580, 150)
done bisec
done bisec
done bisec
done bisec
